# 1D spectral analysis

### Objective
Performing a full spectral anaysis of the Crab pulsar.

### Steps
- Prepare the **data access and selection**
- Set up the **analyis parameters**
  - Define the [reconstructed energy](https://docs.gammapy.org/1.2/user-guide/references.html#term-Reco-Energy) axis and [true energy](https://docs.gammapy.org/1.2/user-guide/references.html#term-True-Energy) axis using the `~gammapy.maps.MapAxis` object
  - Define the spatial geometry
  - Define the exclusion mask
  - Choose the correct `~gammapy.datasets.Dataset` type and define it
- Do the **data reduction**
- Make **the modeling and fitting**
  - Define the `~gammapy.modeling.models.SkyModel` to fit the data. Being this a spectral analysis, the `SkyModel` is completely defined by a  `~gammapy.modeling.models.SpectralModel` (no spatial information is required)
  - Create a `~gammapy.modeling.Fit` object and run it to fit the model parameters
  - Apply a `~gammapy.estimators.FluxPointsEstimator` to compute flux points for the spectral part of the fit.

## Import all the necessary libraries for the tutorial

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion


from gammapy.data import DataStore
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.maps import MapAxis, WcsGeom, RegionGeom
from gammapy.makers import (
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker
)
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
)
from gammapy.visualization import plot_spectrum_datasets_off_regions

In [2]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

#### Exercise:

**Utilise the above `data_store.obs_table` to display only "Crab Nebula" observations using the `OBJECT` column**

In [9]:
selection = data_store.obs_table['OBJECT']==['Crab Nebula']

In [10]:
obs_ids = data_store.obs_table[selection]["OBS_ID"]
observations = data_store.get_observations(obs_ids)

In [11]:
print(observations)

Observations
Number of observations: 4
Observation

	obs id            : 23523 
 	tstart            : 53343.92
	tstop             : 53343.94
	duration          : 1687.00 s
	pointing (icrs)   : 83.6 deg, 21.5 deg

	deadtime fraction : 6.2%
Observation

	obs id            : 23526 
 	tstart            : 53343.95
	tstop             : 53343.97
	duration          : 1683.00 s
	pointing (icrs)   : 83.6 deg, 22.5 deg

	deadtime fraction : 6.6%
Observation

	obs id            : 23559 
 	tstart            : 53345.96
	tstop             : 53345.98
	duration          : 1686.00 s
	pointing (icrs)   : 85.3 deg, 22.0 deg

	deadtime fraction : 6.4%
Observation

	obs id            : 23592 
 	tstart            : 53347.91
	tstop             : 53347.93
	duration          : 1686.00 s
	pointing (icrs)   : 82.0 deg, 22.0 deg

	deadtime fraction : 6.2%



The next step is to define a signal extraction region, also known as `on` region. In the simplest case this is just a CircleSkyRegion.

**Recall our target is the `Crab`.**

In [12]:
target_position = SkyCoord.from_name('Crab')

In [13]:
on_region = CircleSkyRegion(center=target_position, radius=0.11*u.deg)

## Exercise: create a geometry based on the target position, with a bin size of 0.05 and 150 pixel2.

In [ ]:
geom = WcsGeom.create(
    npix=(150, 150), 
    binsz=0.05, 
    skydir=skydir, 
    frame="icrs"
)

We will use the reflected regions method to place off regions to estimate the background level in the on region. To make sure the off regions don’t contain gamma-ray emission, we create an exclusion mask.

Using http://gamma-sky.net/ we find that there’s only one known gamma-ray source near the Crab nebula:

In [ ]:
exclusion_region = CircleSkyRegion(
    center=SkyCoord(183.604, -8.708, unit="deg", frame="galactic"),
    radius=0.5 * u.deg,
)

exclusion_mask = ~geom.region_mask([exclusion_region])
exclusion_mask.plot()

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    0.1, 40, nbin=10, per_decade=True, unit="TeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    0.05, 100, nbin=20, per_decade=True, unit="TeV", name="energy_true"
)

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

Exercise: set up a SpectrumDataset, SpectrumDatasetMaker, ReflectedRegionsBackgroundMaker and SafeMaskMaker

In [ ]:
dataset_empty =
dataset_maker =
bkg_maker =
safe_mask_masker =

In [ ]:
datasets = Datasets()

for obs_id, observation in zip(obs_ids, observations):
    dataset = dataset_maker.run(dataset_empty.copy(name=str(obs_id)), observation)
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    datasets.append(dataset_on_off)

print(datasets)

In [ ]:
ax = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="k")
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)

## Data Fitting

In this section we fit a spectral model to the data. In particular, we can use the [likelihood ratio test](https://docs.gammapy.org/1.2/user-guide/stats/index.html#estimating-ts) to compare two different hypotheses:
- H0: Background only (no source)
- H1: Background + source described by a power law model


### **H0**

The value of the quantity $-2\ln\mathcal(L)$ for the background-only model (null hypothesis) can be simply computed as

In [ ]:
stacked_dataset = datasets.stack_reduce(name='stacked')
print(stacked_dataset)

In [ ]:
Wstat_0 = stacked_dataset.stat_sum()
print(Wstat_0)

Since the background has been estimated using the reflected regions method, here $-2\ln\mathcal(L)$ corresponds to the so-called [Wstat](https://docs.gammapy.org/1.2/user-guide/references.html#term-WStat) fit statistic.

We can inspect the model residuals for the H0 hypothesis:

In [ ]:
stacked_dataset.plot_residuals_spectral();

As expected, the residuals show a clear positive feature indicating that a source is missing in the model.

## **H1**
We now add a source defined by a power law spectrum to the model.

In [ ]:
spectral_model = PowerLawSpectralModel(
    index=2.2, amplitude=2e-11 * u.Unit("1 / (cm2 s TeV)"), reference=1 * u.TeV
)

Define your `SkyModel` and apply it to your `stacked_dataset`.

In [ ]:
model1 =

Create the `Fit` object, and run the fit.

In [ ]:
fit =
result =

In [ ]:
Wstat_1 = result1.total_stat
print(f"delta TS of H1 vs H0: {Wstat_0-Wstat_1}")

In [ ]:
ax_spectrum, ax_residuals = stacked_dataset.plot_fit()